## Segmentation example (nuclear division time series)

In [ ]:
from skimage import io, filters, measure, color, morphology 
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# Read the first tiff stack (Nucseq001.tif)
# These example images are from http://cmci.embl.de
img_stack = io.imread('Nucseq001.tif')
print(img_stack.shape)
number_slices = img_stack.shape[0]
print(f'Time series with {number_slices} slices')

In [ ]:
# Show slice number 23:
plt.imshow(img_stack[23], cmap='gray')

In [ ]:
# Let's threshold the image. What's a suitable threshold?
fig, ax = filters.try_all_threshold(img_stack[23], figsize=(12, 10), verbose=False)
plt.show()

In [ ]:
# Also available: local/adaptive thresholding (you can try this)
# local_threshold = filters.threshold_local(img_stack[0], block_size=1, offset=10)
# binary_img = img_stack[0] > threshold
# plt.imshow(binary_img)

In [ ]:
# Make a copy to store the binary images and keep the original greyscale images
img_stack_binary = np.copy(img_stack)

In [ ]:
# Let's use the Triangle alogrithm for this example
for i in range(number_slices):
    # Find the threshold for each individual slice
    threshold = filters.threshold_triangle(img_stack[i])
    img_stack_binary[i] = img_stack[i] > threshold
    # Also perform binary erosion
    img_stack_binary[i] = morphology.binary_erosion(img_stack_binary[i])

In [ ]:
plt.imshow(img_stack_binary[23], cmap='binary')

In [ ]:
# With ndimage.label we obtain labels of all connected components in the binary image.
img_labels = {}   # empty dictionary
for i in range(number_slices):
    img_labels[i] = ndimage.label(img_stack_binary[i]) 
# This gives us a dictionary that contains the labelled images and number of labelled objects for each slice

In [ ]:
# The returned images have the same dimension as the original images, but each pixel intensity is replaced by the label index
print(img_labels[23])
print('Unique labels in slice 23 (0 is background): ', np.unique(img_labels[23][0]))

In [ ]:
# Show the labels over the original greyscale image for slice 23
img_label_overlay = color.label2rgb(img_labels[23][0], image=img_stack[23], bg_label=0)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img_label_overlay)

In [ ]:
# The function measure.regionprops calculates properties of the labelled regions.
label_properties = {}
for i in range(number_slices):
    label_properties[i] = measure.regionprops(img_labels[i][0])

In [ ]:
# For example, let's look at the centroid coordinates and the areas of all labels in slice 23
# Many more attributes can be accessed, see: 
# https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops

for label in label_properties[23]:
    print(f'The label at centroid coordinates {label.centroid} has an area of {label.area} pixels')

In [ ]:
# How many nuclei are there per slice? How does that number change over time?
cell_count_per_slice = []

for i in range(number_slices):
    cell_counter = 0
    for label in label_properties[i]:
        # Only count label if its area is larger than 20 pixels:
        if label.area > 20:
            cell_counter += 1
    cell_count_per_slice.append(cell_counter)
    
cell_count_per_slice

In [ ]:
# Let's plot that
x = np.arange(number_slices)
y = cell_count_per_slice
plt.plot(x, y)
plt.show()